# Base Score Calculation

## General Approach

In order to understand the performance of our base model we need to calculate a base score. 
The goal of your model is to forecast the category of delay between creation of a shipment and when it will have its `first hub scan`. In other words the target is based on the binned difference between the timestamps `created_at` and `first_hub_scan` of a shipment. We will work with the following bins refering to the number of days between the two timestamps:
- 0 (days)
- 1 (days)
- 2 (days)
- 3 (days)
- 4 (days)
- 5 (days)
- 6 (days)
- 7 (days)

For calculating the base score we will use the average delay between `created` and `first_hub_scan`. 

## Loading and Preparing the Data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dispatcher.data.ticket import Ticket
from dispatcher.data.shipment import Shipment

In [3]:
ticket = Ticket.get_ticket_features(Ticket)
ticket.head()

/Users/christianklaus/code/christianklausML/dispatcher-project/dispatcher/data/ticket.py:24: DtypeWarning: Columns (4,12,22,24,26) have mixed types.Specify dtype option on import or set low_memory=False.
  shipment_order = RawData.get_table_data('shipment', local=True,


ticket features loaded


NAME,First hub scan,International,Order created,Order is being prepared,Order paid,Order shipped,Ordered items are being produced,Refund requested
SHIPMENT_ID,,,,,,,,
168699621.0,2022-03-01 13:30:00,NaT,2022-02-28 21:50:00,NaT,NaT,2022-03-01,NaT,NaT
168699622.0,2022-03-01 15:37:00,NaT,2022-02-28 21:50:00,NaT,NaT,2022-03-01,NaT,NaT
168699623.0,2022-03-01 15:37:00,NaT,2022-02-28 21:50:00,NaT,NaT,2022-03-01,NaT,NaT
168699624.0,2022-03-01 13:19:00,NaT,2022-02-28 21:50:00,NaT,NaT,2022-03-01,NaT,NaT
168699625.0,2022-03-01 18:42:00,NaT,2022-02-28 21:50:00,NaT,NaT,2022-03-01,NaT,NaT


In [4]:
shipment = Shipment.get_shipment_features(Shipment)
shipment.head()

/Users/christianklaus/code/christianklausML/dispatcher-project/dispatcher/data/shipment.py:10: DtypeWarning: Columns (4,12,14) have mixed types.Specify dtype option on import or set low_memory=False.
  clean_shipment = RawData.get_table_data('shipment', local=True, clean=True)


,ID,CARRIER_ID,SHOP_ID,CUSTOMER_ADDRESS_COUNTRY_ID,CUSTOMER_ADDRESS_ZIP_CODE,PLANNED_PICKUP_TIMESTAMP,CREATED_AT,WAREHOUSE_ID,ORDER_ID,SLA_DAYS,RELATION_ID,RELATION_DISTANCE,ORIGIN_ZIP_CODE,ORIGIN_COUNTRY,DESTINATION_ZIP_CODE,DESTINATION_COUNTRY
0,169412848,2,405,216,8400,2022-03-04 17:26:56,2022-03-04 16:26:58,604,151004966,NaN,323,284.537697,96138,DE,9000,CH
1,169265841,2,405,216,1015,2022-03-03 19:23:57,2022-03-03 18:23:58,604,150874683,NaN,323,284.537697,96138,DE,9000,CH
2,170520112,2,405,216,6153,2022-03-11 15:43:42,2022-03-11 14:43:43,604,151958657,NaN,323,284.537697,96138,DE,9000,CH
3,169681312,2,405,216,2300,2022-03-07 16:50:36,2022-03-07 15:50:38,604,151337977,NaN,323,284.537697,96138,DE,9000,CH
4,169641352,2,405,216,6430,2022-03-07 15:09:10,2022-03-07 14:09:12,604,151317317,NaN,323,284.537697,96138,DE,9000,CH


We only need `shipment_id`, `CREATED_AT`, and `First hub scan`.

In [5]:
created = shipment[['ID','CREATED_AT']]
fhs = ticket[['First hub scan']]
fhs.head()

NAME,First hub scan
SHIPMENT_ID,
168699621.0,2022-03-01 13:30:00
168699622.0,2022-03-01 15:37:00
168699623.0,2022-03-01 15:37:00
168699624.0,2022-03-01 13:19:00
168699625.0,2022-03-01 18:42:00


In [6]:
merged = created.merge(fhs, how='left',left_on='ID',right_index=True)
merged.head()

,ID,CREATED_AT,First hub scan
0,169412848,2022-03-04 16:26:58,2022-03-10 16:25:01
1,169265841,2022-03-03 18:23:58,2022-03-09 15:32:34
2,170520112,2022-03-11 14:43:43,2022-03-15 22:19:45
3,169681312,2022-03-07 15:50:38,2022-03-10 15:25:31
4,169641352,2022-03-07 14:09:12,2022-03-10 16:39:10


In [7]:
merged['DIFF_TRUE'] = merged['First hub scan'] - merged['CREATED_AT']
merged['DIFF_TRUE'] = merged['DIFF_TRUE'].astype('timedelta64[D]')
merged.head()

,ID,CREATED_AT,First hub scan,DIFF_TRUE
0,169412848,2022-03-04 16:26:58,2022-03-10 16:25:01,5.0
1,169265841,2022-03-03 18:23:58,2022-03-09 15:32:34,5.0
2,170520112,2022-03-11 14:43:43,2022-03-15 22:19:45,4.0
3,169681312,2022-03-07 15:50:38,2022-03-10 15:25:31,2.0
4,169641352,2022-03-07 14:09:12,2022-03-10 16:39:10,3.0


We exclude all differences that are not in the bins we are interested in. 

In [8]:
clean_df = merged[merged['DIFF_TRUE'].isin([1,2,3,4,5,6,7])].copy()

## Calculating the Average Delay

In [9]:
avg_diff = round(clean_df[['DIFF_TRUE']].mean(),0)
avg_diff

DIFF_TRUE    2.0
dtype: float64

In [10]:
clean_df[['DIFF_PRED']] = avg_diff
clean_df.head()

,ID,CREATED_AT,First hub scan,DIFF_TRUE,DIFF_PRED
0,169412848,2022-03-04 16:26:58,2022-03-10 16:25:01,5.0,2.0
1,169265841,2022-03-03 18:23:58,2022-03-09 15:32:34,5.0,2.0
2,170520112,2022-03-11 14:43:43,2022-03-15 22:19:45,4.0,2.0
3,169681312,2022-03-07 15:50:38,2022-03-10 15:25:31,2.0,2.0
4,169641352,2022-03-07 14:09:12,2022-03-10 16:39:10,3.0,2.0


## Calculating the Base Score `Accuracy`

In [11]:
from sklearn.metrics import accuracy_score

In [12]:
base_score = round(accuracy_score(y_true=clean_df['DIFF_TRUE'], y_pred=clean_df['DIFF_PRED']),2)
base_score

0.24